In [2]:
#Read the base bom files from jde as dataframes

import pandas as pd
import os

outputfolder = r'C:\Users\SPoudel\OneDrive - Ventura Foods\Desktop\Python sample outputs'

# Load the Excel file without headers
jde_bom_us = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset/jde_us.xlsx", 
    skiprows=2
)

jde_bom_can = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset/jde_canada.xlsx", 
    skiprows=2
)

# this completes the horizontals(rows) of data creating needed base Records or Observations
jde_bom = pd.concat([jde_bom_us,jde_bom_can],axis=0,join='outer')
jde_bom.columns = jde_bom.columns.str.replace(" ", "_", regex=True) #replace spaces and blanks with "_"
jde_bom = jde_bom.drop(columns=["3rd_Item_Number", "Extended_Amount"])

jde_bom["Campus_SKU"] = jde_bom["Business_Unit"].astype(str) + "_" + jde_bom["Parent_Item_Number"].astype(str) #creating ref by combining Business Unit and Parent Item Number
jde_bom["Campus_Component"] = jde_bom["Business_Unit"].astype(str) + "_" + jde_bom["Component"].astype(str) #creating component ref by combining business unit and component
jde_bom.columns = ["Campus","Level","Parent_SKU","Parent_SKU_Description","UOM","Component","Component_Description",
                   "Commodity_Class","UM","Stocking_Type","Percent_Scrap","Quantity_Per","Quantity_w_Scrap","Unit_Cost","Campus_SKU","Campus_Component"] #Changing column names to appropriate names

In [56]:
#Sales Order in next 28 days attribute design with location and item# as key
#MODIFICATION: THIS CAN BE MODIFIED WITH NEW DATASET FOR NEXT 28 DAYS OPEN ORDERS AND NO NEED TO GET CANADA OPEN ORDERS
custorder28days = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset/US and CAN OO BT where status _ J.xlsx",skiprows=2)
custorder28days['Product Label (SKU) Code'].replace("-","",regex=True,inplace=True)
custorder28days['Location_SKU'] = custorder28days['Location No'].astype(str) + '_' + custorder28days['Product Label (SKU) Code'].astype(str)
custorder28days['Campus_SKU'] = custorder28days['Campus No.'].astype(str) + '_' + custorder28days['Product Label (SKU) Code'].astype(str)
custorder28days = custorder28days.iloc[1:]
#custorder28days.to_excel(os.path.join(outputfolder,"custord28days.xlsx"), index=False)

c:\Anaconda_Distribution\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\SPoudel\AppData\Local\Temp\ipykernel_27036\2564013234.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  custorder28days['Product Label (SKU) Code'].replace("-","",regex=True,inplace=True)


In [58]:
#Inventory With Hold Status for FG and RM
InvWHoldStatus= pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset/InventoryWCampusForBOM_FROMObject.xlsx",skiprows=2)

#InvWHoldStatus.columns = ['Campus','Location','SKU','Item_Type','Hard_Hold','Soft_Hold','Useable']
InvWHoldStatus_FG = InvWHoldStatus[InvWHoldStatus['Item_Type'] == 'FG']
InvWHoldStatus_RM = InvWHoldStatus[InvWHoldStatus['Item_Type'] == 'COMP']
InvWHoldStatus_WIP = InvWHoldStatus[InvWHoldStatus['Item_Type'] == 'WIP']


c:\Anaconda_Distribution\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [107]:
LabelInventory25 = InvWHoldStatus= pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset/JDE Inventory Lot Detail.xlsx",skiprows=6)
LabelInventory25 = LabelInventory25[(LabelInventory25['MPF'] == 'LBL') & (LabelInventory25['BP'] == 25)][['BP','Item Number','Status','On Hand']]
LabelInventory25

,BP,Item Number,Status,On Hand
7103,25,84416,NaN,5573.8
7104,25,85610,NaN,18406.9
7105,25,86083,NaN,13531.2
7106,25,86388,NaN,5901.4
7107,25,86888,NaN,36000.0
...,...,...,...,...
7577,25,99552,NaN,12530.9
7578,25,99553,NaN,8845.9
7579,25,99558,NaN,7586.9
7580,25,99560,NaN,8000.0


In [ ]:
#reading Lot status Lookup
LotStatusCode = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Static_MonthlyLookup Dataset/Lot Status Code.xlsx")
LabelInventory25WHoldStatus =  LabelInventory25.merge(LotStatusCode[['Lot status','Hard/Soft Hold']],how='left',left_on='Status',right_on='Lot status')[['BP','Item Number','On Hand','Hard/Soft Hold']]
LabelInventory25WHoldStatus['Hard/Soft Hold'] =LabelInventory25WHoldStatus['Hard/Soft Hold'].fillna('Useable')

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

#Forecast Dataset
ForecastLatestCopy= pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Weekly Dataset/02.11.2025/DSX Forecast Backup - 2025.02.11.xlsx",skiprows=2)
ForecastLatestCopy['Forecast Month Year ID'] = pd.to_datetime(ForecastLatestCopy["Forecast Month Year ID"], format="%Y%m") + pd.offsets.MonthBegin(0)

today = pd.to_datetime("today")
ForecastLatestCopy['MonthFromCurrent'] = (ForecastLatestCopy['Forecast Month Year ID'].dt.year - today.year)*12 + (ForecastLatestCopy['Forecast Month Year ID'].dt.month - today.month)
ForecastLatestCopy['Product Label (SKU) Code'] = ForecastLatestCopy['Product Label (SKU) Code'].str.replace('-','') #change SKU name and without - in middle
ForecastLatestCopy['ManufLocationCampus_Item'] = ForecastLatestCopy['Product Manufacturing Location Code'].astype(str) + "_" + ForecastLatestCopy['Product Label (SKU) Code'].astype(str) #Create Key (manf. Loc. acts as Campus and SKU) to join with other dataframes
#ForecastLatestCopy['Adjusted Forecast Cases'] = ForecastLatestCopy['Adjusted Forecast Cases'].fillna(0)

FcstNxt2Mths = ForecastLatestCopy[(ForecastLatestCopy['MonthFromCurrent'] < 2) & (ForecastLatestCopy['MonthFromCurrent'] >= 0)]
FcstNext6Months = ForecastLatestCopy[(ForecastLatestCopy['MonthFromCurrent'] < 6) & (ForecastLatestCopy['MonthFromCurrent'] >= 0)]
FcstNext12Months = ForecastLatestCopy[(ForecastLatestCopy['MonthFromCurrent'] < 12) & (ForecastLatestCopy['MonthFromCurrent'] >= 0)]

FcstNxt2MthsPivot =  FcstNxt2Mths.pivot_table(values='Adjusted Forecast Cases',index='ManufLocationCampus_Item',columns='MonthFromCurrent',aggfunc='sum').reset_index()
FcstNxt2MthsPivot.rename(columns={0:'ThisMonthFcst',1:'NextMonthFcst'}, inplace=True)

FcstNext6MonthsPivot =  FcstNext6Months.pivot_table(values='Adjusted Forecast Cases',index='ManufLocationCampus_Item',aggfunc='sum').reset_index()
FcstNext6MonthsPivot.rename(columns={'Adjusted Forecast Cases':'Next6MonthsFcst'}, inplace=True)
FcstCondensed =   FcstNxt2MthsPivot.merge(FcstNext6MonthsPivot,how='outer')

c:\Anaconda_Distribution\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
